In [1]:
import pickle
import pandas as pd

model_path = open('model/model_regression.pkl', 'rb')
model = pickle.load(model_path)
model

DecisionTreeRegressor()

In [2]:
df = pd.read_csv('data/MSFT_Processed_Data.csv', parse_dates=['Date'], index_col=['Date'])
df.head()

,Open,High,Low,Close,Adj Close,Volume,change_tmr_%,Tmr_Projection
Date,,,,,,,,
2016-12-09,61.180000,61.990002,61.130001,61.970001,56.274216,27349400,0.321694,UP
2016-12-12,61.820000,62.299999,61.720001,62.169998,56.455826,20198100,1.286125,UP
2016-12-13,62.500000,63.419998,62.240002,62.980000,57.191376,35718900,-0.478620,DOWN
2016-12-14,63.000000,63.450001,62.529999,62.680000,56.918953,30352700,-0.159793,DOWN
2016-12-15,62.700001,63.150002,62.299999,62.580002,56.828133,27669900,-0.449442,DOWN


In [3]:
from backtesting import Backtest, Strategy

In [4]:
class Regression(Strategy):
    limit_buy = 1
    limit_sell = -5
    def init(self):
        self.model = model
        self.already_bought = False
    
    def next(self):
        explanatory_today = self.data.df.tail(1)
        forecast_tmr = model.predict(explanatory_today)[0]
        if forecast_tmr > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tmr < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

In [5]:
df_explanatory = df.drop(columns=["change_tmr_%",'Adj Close',	"Tmr_Projection"])

In [6]:
bt = Backtest(df_explanatory, Regression, cash=10000, commission=0.002, exclusive_orders=True)

In [7]:
list_limit_buy = list(range(0,11,1))
list_limit_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [8]:
list_limit_sell = list(range(0,-11,-1))
list_limit_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [9]:
%%time
results = bt.optimize(limit_buy = list_limit_buy, limit_sell = list_limit_sell, maximize= 'Return [%]', return_heatmap=True)

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: total: 2min 27s
Wall time: 5min 36s


In [23]:
dff = results[1].reset_index()

In [26]:
dff = dff.pivot(index = 'limit_buy',columns='limit_sell', values='Return [%]')

In [28]:
dff.sort_index(axis=1, ascending=False).style.format(precision=0).background_gradient(vmin=dff.values.min(), vmax= dff.values.max())

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,48448665,11918852,351719,31166,4397,1180,704,569,548,524,524
1,4410709,3889098,398948,29952,4771,1393,704,569,548,524,524
2,17533,27160,27887,13120,4014,1266,704,569,548,523,523
3,168,224,276,244,1469,662,410,324,379,361,361
4,-61,-53,-49,30,937,578,370,291,342,361,361
5,-79,-80,-76,-72,117,320,194,291,342,361,361
6,-100,-85,-82,-81,48,296,194,291,342,361,361
7,-100,-100,-100,-100,-100,43,407,417,312,329,329
8,-100,-100,-100,-100,-100,-100,-100,-100,-100,158,158


In [10]:
result = bt.run()

In [11]:
result.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-09 00:00:00
End,2024-09-09 00:00:00
Duration,2831 days 00:00:00
Exposure Time [%],99.897331
Equity Final [$],149275.281624
Equity Peak [$],171352.404305
Return [%],1392.752816


In [12]:
bt.plot(filename='backtesting_report/regression.html')

Row(id='1407', ...)